# trying out signatures

In [1]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256
from Crypto.Signature import DSS

def make_keypair():
    privkey = ECC.generate(curve='P-256')
    pubkey = privkey.public_key()
    return privkey, pubkey 

def make_hex_key(key):
    binary = key.public_key().export_key(format='DER')
    return binary.hex()

def sign_something(priv_key, something):
    signer = DSS.new(priv_key, 'fips-186-3')
    h = SHA256.new(something)
    signature = signer.sign(h)
    return signature.hex()

def verify_signature(pub_key, something, signature):
    h = SHA256.new(something)
    verifier = DSS.new(pub_key, 'fips-186-3')
    try:
        verifier.verify(h, signature)
        print ("The message is authentic.")
    except ValueError:
        print ("The message is not authentic.")

priv_key, pub_key = make_keypair()
priv_key2, pub_key2 = make_keypair()

message = b'I accept this'
signature = sign_something(priv_key, message)
bitas = bytes.fromhex(signature)
verify_signature(pub_key, message, bitas)

hexkey = make_hex_key(pub_key)
hexkey

The message is authentic.


'3059301306072a8648ce3d020106082a8648ce3d03010703420004f2e282c65c5ca62f3515526822376748662fc5baad5df500ead3926536142feb92494b03cad28e72ee265bbf2502ec902ef052892380794fca1f05213efc0109'

# trying out transactions

In [2]:
from transaction import Transaction, TxIn, TxOut

COINBASE_AMOUNT = 20

bob_addr = "bob123"
alice_addr = "alice321"
bob_privkey, bob_pubkey = make_keypair()

# f = open('privatekey.der', 'wt')
# f.write(bob_privkey.export_key(format='DER').hex())
# f.close()

# Coinbase transaction to Bob
coinbase_in = TxIn("", 0, "")
coinbase_out = TxOut(bob_addr, COINBASE_AMOUNT)
coinbase_transaction = Transaction([coinbase_in], [coinbase_out])

# Bob's gift to Alice
bob_in = TxIn(coinbase_transaction.id, 0, "")
bob_out1 = TxOut(alice_addr, 10)
bob_out2 = TxOut(bob_addr, 10)
bobs_gift_transaction = Transaction([bob_in], [bob_out1, bob_out2])

unsignedTxIns = bobs_gift_transaction.txIns
for uTxIn in unsignedTxIns:
    signature = sign_something(bob_privkey, bobs_gift_transaction.id.encode())
    uTxIn.signature = signature

# Verifying that thing is signed
signa = bytes.fromhex(unsignedTxIns[0].signature)
verify_signature(bob_pubkey, bobs_gift_transaction.id.encode(), signa)

The message is authentic.


# blockchain node toy model 

In [3]:
from blockchain import Blockchain, Block
from transaction import *

blockchain = Blockchain()
unspent_txouts = []

block_transactions = blockchain.last_block.data

unspent_txouts = update_unspent_txouts(block_transactions, unspent_txouts)

unspent_txouts

[{'txOutId': '9b77747f8cddb3e78b32c75cbcfe6cccf4544a4fc5886409b02865e09db21875', 'txOutIndex': 0, 'address': '3059301306072a8648ce3d020106082a8648ce3d0301070342000447e45d57be90ac1ad97c5232f7922ade8e63f9931666f1fab5f1bcaac0bdc4b83bf68c72534eb33b68865b5e5465c7ba46094961b578513aed09ed22ab8c03f3', 'amount': 50}]